In [75]:
import chromadb
import json

from chromadb.utils import embedding_functions

# Read menu json.
with open('short_menu.json', 'r') as f:
    menu = json.load(f)

### Generate embeddings for menu items

### Prepare the data for chroma db.

In [110]:
transformer = embedding_functions.SentenceTransformerEmbeddingFunction(model_name='bert-base-nli-mean-tokens')

client = chromadb.Client()

texts = []
metadata = []
ids = []

for category in menu['data']['menuCategories']:
    for item in category['items']:
        texts.append(f"Name: {item['name']} | Price: {item['price']}")
        metadata.append({
            "id": item['id'],
            "name": item['name'],
            "price": item['price'],
            "category": category['menuCategoryName'],
            "hotel": item['hotel']
        })
        ids.append(str(item['id']))


client.delete_collection('menu_collection')

collection = client.get_or_create_collection('menu_collection', embedding_function=transformer)

## Recommendation System

In [111]:
# Add data to collection
collection.add(ids=ids, documents=texts, metadatas=metadata)

In [118]:
results = collection.query(
    n_results=2,
    query_texts=["List hotel 100 foods"],
)

results

{'ids': [['259', '279']],
 'embeddings': None,
 'documents': [['Name: Pizza | Price: 100.0',
   'Name: Chocolate Brownie (BQT) | Price: 125.0']],
 'uris': None,
 'data': None,
 'metadatas': [[{'category': "Today's Offers",
    'hotel': 34,
    'id': 259,
    'name': 'Pizza',
    'price': 100.0},
   {'category': "Today's Offers",
    'hotel': 34,
    'id': 279,
    'name': 'Chocolate Brownie (BQT)',
    'price': 125.0}]],
 'distances': [[155.05453491210938, 270.24859619140625]],
 'included': [<IncludeEnum.distances: 'distances'>,
  <IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}